In [2]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

In [3]:
new_record = []
for seq_record in SeqIO.parse('/home/ouconstand/data/Virus_Host/seq_data/sequence_0.fasta','fasta'):
    seq = seq_record.seq
    if len(seq) > 2000:
        for i in range(0, len(seq), 500):
            if i + 2000 > len(seq):
                new_seq = seq[-2000:]
                rec = SeqRecord(new_seq, id=seq_record.id, description=seq_record.description, name=seq_record.name)
                new_record.append(rec)
                break

            new_seq = seq[i:i+2000]
            rec = SeqRecord(new_seq,id=seq_record.id,description=seq_record.description,name=seq_record.name)
            new_record.append(rec)

        else:
            print('sequence length is less than 2000')
            print(seq_record.description)

In [4]:
data = [i.seq for i in new_record]
labels = [i.id for i in new_record]

In [5]:
set(labels)

{'KT368856.1', 'KT368895.1', 'MT726043.1', 'MZ519413.1'}

In [6]:
import numpy as np

def nucl_to_onehot(sequence, dimension=5):
    __nucl_list = ['A', 'T', 'C', 'G', 'N']
    __nucl_to_int = dict((nucl, int) for int, nucl in enumerate(__nucl_list))
    __integer_encoded = [__nucl_to_int[base] for base in sequence]

    result = np.zeros((len(sequence), dimension))
    for i, base in enumerate(__integer_encoded):
        result[i, base] = 1
    return result


In [13]:
labels_to_int = dict((int, label) for label, int in enumerate(set(labels)))
process_data = np.array([nucl_to_onehot(i) for i in data])
process_labels = np.array([labels_to_int[label] for label in labels])

In [29]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(512,activation='relu', input_shape=(5,2000)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [30]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model.fit(process_data,process_labels,epochs=5, batch_size=128)

Epoch 1/5


ValueError: in user code:

    /home/ouconstand/software/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/ouconstand/software/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/ouconstand/software/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ouconstand/software/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ouconstand/software/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
   

In [36]:
process_data[0]

array([[1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]])